In [0]:
!pip install OpenNMT-py

In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py && python setup.py install

In [1]:
!unzip Corpora.zip

Archive:  Corpora.zip
   creating: Corpora/
   creating: Corpora/En2Fa-Translation/
   creating: Corpora/En2Fa-Translation/Dev/
  inflating: Corpora/En2Fa-Translation/Dev/dev.en  
  inflating: Corpora/En2Fa-Translation/Dev/dev.fa  
   creating: Corpora/En2Fa-Translation/Test/
  inflating: Corpora/En2Fa-Translation/Test/test.en  
  inflating: Corpora/En2Fa-Translation/Test/test.fa0  
  inflating: Corpora/En2Fa-Translation/Test/test.fa1  
  inflating: Corpora/En2Fa-Translation/Test/test.fa2  
  inflating: Corpora/En2Fa-Translation/Test/test.fa3  
   creating: Corpora/En2Fa-Translation/Train/
  inflating: Corpora/En2Fa-Translation/Train/train.en  
  inflating: Corpora/En2Fa-Translation/Train/train.fa  
   creating: Corpora/Transliteration/
  inflating: Corpora/Transliteration/dev.en  
  inflating: Corpora/Transliteration/dev.fa  
  inflating: Corpora/Transliteration/test.en  
  inflating: Corpora/Transliteration/test.fa  
  inflating: Corpora/Transliteration/train.en  
  inflating: Corpor

In [2]:
!onmt_preprocess -train_src Corpora/En2Fa-Translation/Train/train.en -train_tgt Corpora/En2Fa-Translation/Train/train.fa -valid_src Corpora/En2Fa-Translation/Dev/dev.en -valid_tgt Corpora/En2Fa-Translation/Dev/dev.fa -save_data ./demo

[2020-06-02 17:36:12,183 INFO] Extracting features...
[2020-06-02 17:36:12,183 INFO]  * number of source features: 0.
[2020-06-02 17:36:12,183 INFO]  * number of target features: 0.
[2020-06-02 17:36:12,183 INFO] Building `Fields` object...
[2020-06-02 17:36:12,183 INFO] Building & saving training data...
[2020-06-02 17:36:12,226 INFO] Building shard 0.
[2020-06-02 17:36:13,201 INFO]  * saving 0th train data shard to ./demo.train.0.pt.
[2020-06-02 17:36:13,906 INFO]  * tgt vocab size: 5908.
[2020-06-02 17:36:13,910 INFO]  * src vocab size: 3116.
[2020-06-02 17:36:13,970 INFO] Building & saving validation data...
[2020-06-02 17:36:13,999 INFO] Building shard 0.
[2020-06-02 17:36:14,006 INFO]  * saving 0th valid data shard to ./demo.valid.0.pt.


In [3]:
!onmt_train -data demo -save_model demo-model -train_steps 50000 -save_checkpoint_steps 50000 -valid_steps 1200 -world_size 1 -gpu_rank 0 -tgt_word_vec_size 1000

[2020-06-02 17:36:18,379 INFO]  * src vocab size = 3116
[2020-06-02 17:36:18,379 INFO]  * tgt vocab size = 5908
[2020-06-02 17:36:18,379 INFO] Building model...
[2020-06-02 17:36:27,123 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(3116, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(5908, 1000, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1500, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (l

In [4]:
!onmt_translate -model demo-model_step_50000.pt -src Corpora/En2Fa-Translation/Test/test.en -output pred.txt -replace_unk -verbose

[2020-06-02 18:46:19,316 INFO] Translating shard 0.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.

SENT 1: ['hello', ',', 'do', 'we', 'drive', 'together', 'to', 'Hanover', 'on', 'the', 'twenty-eighth', 'of', 'March', '?']
PRED 1: سلام ، ما رانندگی با هم در سفر در فرودگاه مارچ ؟
PRED SCORE: -3.0671

SENT 2: ['it', 'is', 'more', 'comfortable', 'by', 'train', '.']
PRED 2: دفتر من بیشتری است .
PRED SCORE: -3.9149

SENT 3: ['do', 'you', 'go', 'by', 'car', 'and', 'I', 'go', 'by', 'train', '?']
PRED 3: آیا شما به ماشین شرکت من با قطار برویم ؟
PRED SCORE: -2.8021

SENT 4: ['I', 'would', 'like', 'to', 'go', 'by', 'train', '.', 'and', 'what', 'would', 'you', 'like', '?']
PRED 4: من مایلم با قطار . و شما چه میخواهید ؟
PRED SCORE: -2.0859

SENT 5: ['if', 'we', 'take', 'the', '$I-$C-$E', 'train', 'at',

In [5]:
!perl OpenNMT-py/tools/multi-bleu.perl Corpora/En2Fa-Translation/Test/test.fa0 < pred.txt

BLEU = 15.64, 56.7/24.6/11.5/4.7 (BP=0.942, ratio=0.943, hyp_len=2539, ref_len=2692)


In [6]:
!perl OpenNMT-py/tools/multi-bleu.perl Corpora/En2Fa-Translation/Test/test.fa1 < pred.txt

BLEU = 17.60, 57.9/26.3/13.8/6.0 (BP=0.934, ratio=0.936, hyp_len=2539, ref_len=2713)


In [7]:
!perl OpenNMT-py/tools/multi-bleu.perl Corpora/En2Fa-Translation/Test/test.fa2 < pred.txt

BLEU = 13.36, 51.0/19.7/9.4/4.3 (BP=0.939, ratio=0.940, hyp_len=2539, ref_len=2700)


In [8]:
!perl OpenNMT-py/tools/multi-bleu.perl Corpora/En2Fa-Translation/Test/test.fa3 < pred.txt

BLEU = 14.83, 50.9/21.2/10.3/4.6 (BP=0.986, ratio=0.986, hyp_len=2539, ref_len=2575)
